## Анализ логов продуктового магазина

Представьте, что вы работаете аналитиком онлайн-магазина и ваша цель - повысить GMV (суммарную выручку от продажи товаров). Для этого вы хотите провести ряд исследований и придумать какие-то меры для увеличения выручки.

В файле <i>orders_with_items.json</i> представлена информация о заказах в онлайн-магазине.
```
OrderID - ID Заказа,
UserID - ID клиента,
OrderTime - время заказа,
items - список элементов в заказе, каждый элемент описан словарем:
    ItemID  - ID продукта, 
    price - цена единицы товара, 
    quantity - количество единиц товара в заказе
```
В файле <i>items_with_categ.csv</i> представлена информация о категории товаров.
```
ItemID - ID продукта,
Category - категория продукта,
ParentCategory - более крупная категория продукта
```

1. Для начала поймем, какого числа было сделано больше всего заказов?
2. Давайте посмотрим на категории товаров. Найдите топ-3 ParentCategory по GMV (суммарная выручка, которую принесла продажа товаров этой категории)?
3. Представьте, что мы хотим запустить какую-то акцию для клиентов или предложить какую-то бонусную программу. Какую акцию было бы целесообразно провести и на каких клиентов её ориентировать? Составьте какой-то список клиентов, которым вы бы предложили участие в акции/программе.
4. Представьте, что для повышения GMV мы хотим внедрить на сайте систему рекомендаций и предлагать пользователю какие-то товары по мере сбора корзины. Что бы вы порекомендовали пользователю X?

Если для решения задачи вам потребуются какие-то сторонние библиотеки, вы можете их установить. __[Документация по установке пакетов](https://cloud.yandex.ru/docs/datasphere/operations/projects/install-dependencies)__

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [39]:
df_cat = pd.read_csv("items_with_categ.csv")
df_cat.head()

,ItemID,Category,ParentCategory
0,1000002,Сладости для диабетиков,Сладости
1,1000046,Вода,Напитки
2,1000048,Сладости для диабетиков,Сладости
3,1000049,Вода,Напитки
4,1000061,Фрукты,Овощи и фрукты


In [5]:
import json
with open('orders_with_items.json') as jf:
    data_js = json.load(jf)

In [12]:
df = pd.DataFrame(data_js)
df.head()

,OrderID,UserID,OrderTime,items
0,90659865,1609459696627452541,2021-01-01 03:51:42,"[{'ItemID': '413736', 'price': 36.72, 'quantit..."
1,90622054,2499292651609021773,2021-01-01 04:13:13,"[{'ItemID': '961454', 'price': 179.9, 'quantit..."
2,90659952,394586611473861433,2021-01-01 05:48:53,"[{'ItemID': '306553', 'price': 101.9, 'quantit..."
3,89765281,5412764681526420705,2021-01-01 08:33:41,"[{'ItemID': '979703', 'price': 99.9, 'quantity..."
4,90582308,6380060821544787608,2021-01-01 09:55:35,"[{'ItemID': '303765', 'price': 44.9, 'quantity..."


In [15]:
df['items'][0][:3]

[{'ItemID': '413736', 'price': 36.72, 'quantity': 2},
 {'ItemID': '523171', 'price': 90.32, 'quantity': 1},
 {'ItemID': '452831', 'price': 223.11, 'quantity': 1}]

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40375 entries, 0 to 40374
Data columns (total 4 columns):
OrderID      40375 non-null object
UserID       40375 non-null object
OrderTime    40375 non-null object
items        40375 non-null object
dtypes: object(4)
memory usage: 1.2+ MB


In [20]:
df['date'] = pd.to_datetime(df.OrderTime).dt.date

In [27]:
df.groupby('date')['OrderID'].count().sort_values()[::-1][0]

342

In [28]:
df.head()

,OrderID,UserID,OrderTime,items,date
0,90659865,1609459696627452541,2021-01-01 03:51:42,"[{'ItemID': '413736', 'price': 36.72, 'quantit...",2021-01-01
1,90622054,2499292651609021773,2021-01-01 04:13:13,"[{'ItemID': '961454', 'price': 179.9, 'quantit...",2021-01-01
2,90659952,394586611473861433,2021-01-01 05:48:53,"[{'ItemID': '306553', 'price': 101.9, 'quantit...",2021-01-01
3,89765281,5412764681526420705,2021-01-01 08:33:41,"[{'ItemID': '979703', 'price': 99.9, 'quantity...",2021-01-01
4,90582308,6380060821544787608,2021-01-01 09:55:35,"[{'ItemID': '303765', 'price': 44.9, 'quantity...",2021-01-01


In [33]:
type(df['items'][0][0])

dict

In [47]:
def item_collector(items):
    items_single = []
    for item in items:
        items_single.append(item['ItemID'])
    return items_single

df['items_list'] = df['items'].apply(item_collector)

In [57]:
cat_map = {k:v for k,v in zip(df_cat['ItemID'],df_cat['Category'])}

In [50]:
items_id = []
for values in df['items_list']:
    items_id.extend(values)

In [54]:
items_id[:3]

['413736', '523171', '452831']

In [53]:
from collections import Counter
Counter(items_id).most_common(3)

[('303744', 6086), ('430560', 5068), ('305660', 3792)]

In [59]:
cat_map[303744],cat_map[430560], cat_map[305660]

('Овощи', 'Фрукты', 'Сахар')

In [109]:
# df['items']

In [114]:
items = {}
for item in df['items']:
    for i in item:
        id_ = i['ItemID']
        if id_ in items:
            items[id_] += i['price']
        else:
            items[id_] = i['price']  


In [91]:
list(cat_map.items())[:2]

[(1000002, 'Сладости для диабетиков'), (1000046, 'Вода')]

In [93]:
count_cats = {}
for id_, price in items.items():
    if cat_map[int(id_)] not in count_cats:
        count_cats[cat_map[int(id_)]] = price
    else:
        count_cats[cat_map[int(id_)]] += price

In [96]:
cat_map_p = {k:v for k,v in zip(df_cat['ItemID'],df_cat['ParentCategory'])}

In [105]:
list(items)[:3]

['413736', '523171', '452831']

In [117]:
count_cats_p = {}
for id_, price in items.items():
#     print(cat_map_p[int(id_)])
    if cat_map_p[int(id_)] not in count_cats_p:
        count_cats_p[cat_map_p[int(id_)]] = price
    else:
        count_cats_p[cat_map_p[int(id_)]] += price

In [118]:
{k: v for k, v in sorted(count_cats_p.items(), key=lambda item: item[1],reverse=True)}

{'Мясо, птица, рыба': 25027932.55999977,
 'Овощи и фрукты': 20471893.579999816,
 'Бакалея': 17280428.949999828,
 'Молочная продукция': 15046480.350000117,
 'Напитки': 7158028.930000112,
 'Красота, гигиена, здоровье': 6075844.6900002295,
 'Бытовая химия': 5602939.980000075,
 'Мучные изделия': 5209598.220000051,
 'Сладости': 3682739.8999999724,
 'Зоотовары': 2180066.7499999776,
 'Для детей': 1227599.399999999,
 'Для дома и сада': 1145950.5500000017,
 'Кухонные принадлежности': 920908.9700000039,
 'Для праздника': 236968.48999999987,
 'Текстиль': 132118.37999999977,
 'Системы нагревания': 125732.0,
 'Для школы': 17424.45999999999}

In [99]:
list({k: v for k, v in sorted(count_cats.items(), key=lambda item: item[1],reverse=True)})[:3]

['Сухие корма', 'Кофе', 'Сыры']

In [101]:
df_cat['ParentCategory'].unique()

array(['Сладости', 'Напитки', 'Овощи и фрукты',
       'Красота, гигиена, здоровье', 'Для праздника',
       'Кухонные принадлежности', 'Бакалея', 'Молочная продукция',
       'Мясо, птица, рыба', 'Мучные изделия', 'Зоотовары',
       'Для дома и сада', 'Бытовая химия', 'Для детей',
       'Системы нагревания', 'Текстиль', 'Для школы'], dtype=object)